### How to work with Chormadb and store it in a database

- Here we insert docs into db
- Uses search operation on it
- Get the context
- Provide that context to the LLM with scores
- Generate the response

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
# langchain + chormadb
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

#for chromadb
from langchain_community.vectorstores import Chroma
from typing import List
import os



In [5]:
strings_array = [
    "Python is renowned for its flexible data structures, which include lists, tuples, sets, and dictionaries. Lists are ordered collections that support dynamic resizing and a variety of methods for adding, removing, or searching elements. Tuples are similar to lists but immutable, ensuring that data remains unchanged once assigned and making them suitable for fixed data groupings. Dictionaries use key-value pairs allowing fast access, manipulation, and association of data by unique keys. Sets are collections of unordered, unique elements, great for removing duplicates and performing mathematical operations like unions and intersections. These data structures form the backbone for efficient algorithm development, making Python popular for data engineering, scientific computing, and rapid prototyping in diverse software projects."
    ,
    "Docker containers have revolutionized application deployment by encapsulating all dependencies within lightweight, portable units. The container lifecycle runs through image creation, build, run, and destruction, allowing consistency across environments from a developer’s laptop to cloud production servers. Networking and persistent storage are handled through Docker’s bridge networks and mounted volumes, often configured in a YAML file for orchestration. Security features, resource limits, and automated health checks help maintain uptime and isolation. Command-line tools and APIs provide granular control, while platforms like Kubernetes extend management to large-scale clusters. Docker’s architecture enables microservices, CI/CD pipelines, and efficient scaling for modern software infrastructure."
    ,
    "Vector databases have emerged as critical infrastructure for AI-driven applications by enabling the fast, approximate search of high-dimensional embeddings. Unlike traditional relational stores, vector DBs represent data points as mathematical vectors, supporting similarity queries, KNN search, and clustering. This approach underpins semantic retrieval in RAG systems, recommendation engines, and fraud detection. Technologies such as Pinecone, Weaviate, and ChromaDB offer APIs to store, update, and query embeddings generated by models like BERT or CLIP. They optimize for speed and scalability with techniques including approximate neighbors, distributed indexing, and GPU acceleration. Advanced filtering and metadata support enable hybrid retrieval for context-aware generative AI solutions."
    ,
    "SQL query optimization is crucial for scalable database operations. Indexing frequently searched columns is an essential strategy, but too many indexes can degrade write performance. Queries should avoid ‘SELECT *’, minimize joins to essential tables, and use WHERE clauses that leverage indexed columns. Tools like EXPLAIN PLAN visualize execution steps, guiding developers to restructure queries for efficiency. Partitioning large tables can improve access speed while reducing locking contention. Regularly updating table statistics ensures the optimizer selects the best execution path. Avoiding correlated subqueries and using batch processing techniques can reduce resource consumption. These approaches together lead to faster, more reliable database systems."
    ,
    "In Node.js, the event loop is a key mechanism that allows non-blocking I/O operations on a single thread. Each incoming request is delegated to the system kernel, freeing the JavaScript runtime to handle other events. Async callbacks are queued and executed when the kernel signals completion, drastically improving throughput. Promises and async/await syntax further simplify asynchronous code management, reducing callback hell and enhancing maintainability. Node.js excels in microservices, web servers, and real-time applications like chat or streaming services. Its event-driven model, combined with fast V8 execution, supports horizontal scaling and resource-efficient concurrency on modest hardware, making Node.js immensely popular for backend services."
]


In [6]:
# Storing the samlpes in a file
import tempfile
temp_dir = tempfile.mkdtemp()

for i,doc in enumerate(strings_array):
    file_path = os.path.join(temp_dir, f"doc_{i}.txt")
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(doc)

print("Doc created")

Doc created


In [8]:

# Document loading
from langchain_community.document_loaders import DirectoryLoader,TextLoader
load = DirectoryLoader(
    temp_dir,
    glob="*.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)

documents=load.load()
for i,doc in enumerate(documents):
    print(f"Document {i}:\n{doc.page_content}")
    print(f"Metadata: {doc.metadata}")

Document 0:
Python is renowned for its flexible data structures, which include lists, tuples, sets, and dictionaries. Lists are ordered collections that support dynamic resizing and a variety of methods for adding, removing, or searching elements. Tuples are similar to lists but immutable, ensuring that data remains unchanged once assigned and making them suitable for fixed data groupings. Dictionaries use key-value pairs allowing fast access, manipulation, and association of data by unique keys. Sets are collections of unordered, unique elements, great for removing duplicates and performing mathematical operations like unions and intersections. These data structures form the backbone for efficient algorithm development, making Python popular for data engineering, scientific computing, and rapid prototyping in diverse software projects.
Metadata: {'source': 'C:\\Users\\samantap\\AppData\\Local\\Temp\\tmpkx50hwz6\\doc_0.txt'}
Document 1:
Docker containers have revolutionized application

### Text Splitting from docs


In [9]:
# Text Splitting
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=20,
    length_function=len,
    separators=[" "]
)
chunks=splitter.split_documents(documents)
print(f"Total chunks: {len(chunks)} of {len(documents)}")
print(f"Content: {chunks[0]}")
print(f"Metadata: {chunks[0].metadata}")

Total chunks: 15 of 5
Content: page_content='Python is renowned for its flexible data structures, which include lists, tuples, sets, and dictionaries. Lists are ordered collections that support dynamic resizing and a variety of methods for adding, removing, or searching elements. Tuples are similar to lists but immutable, ensuring that data' metadata={'source': 'C:\\Users\\samantap\\AppData\\Local\\Temp\\tmpkx50hwz6\\doc_0.txt'}
Metadata: {'source': 'C:\\Users\\samantap\\AppData\\Local\\Temp\\tmpkx50hwz6\\doc_0.txt'}


### Embedding

In [10]:
sample_text = "The quick brown fox jumps over the lazy dog."
embeddings=HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"

)
embeddings.embed_query(sample_text)

[0.043933555483818054,
 0.05893440172076225,
 0.04817839711904526,
 0.07754813879728317,
 0.02674437128007412,
 -0.03762954846024513,
 -0.0026051148306578398,
 -0.05994303897023201,
 -0.0024960089940577745,
 0.02207283116877079,
 0.048025909811258316,
 0.055755313485860825,
 -0.03894546255469322,
 -0.026616770774126053,
 0.0076933917589485645,
 -0.026237700134515762,
 -0.03641606494784355,
 -0.03781614825129509,
 0.07407816499471664,
 -0.04950505867600441,
 -0.05852171406149864,
 -0.06361967325210571,
 0.032435014843940735,
 0.022008540108799934,
 -0.07106371223926544,
 -0.03315779194235802,
 -0.06941041350364685,
 -0.05003739148378372,
 0.07462679594755173,
 -0.11113381385803223,
 -0.01230629812926054,
 0.03774565830826759,
 -0.02803134173154831,
 0.014535323716700077,
 -0.031558554619550705,
 -0.08058364689350128,
 0.05835256725549698,
 0.002590067917481065,
 0.0392802357673645,
 0.025769580155611038,
 0.049850545823574066,
 -0.0017562442226335406,
 -0.04552978649735451,
 0.029260773

#### Storing into ChromaDB using HuggingFace Space

In [11]:
# Directory of ChromaDB
persistant_directory = "./chromaDB"

VECTOR_STORE = Chroma(
    persist_directory=persistant_directory,
    embedding_function=HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    ),
    collection_name="Rag_collection"
)
VECTOR_STORE.add_documents(chunks)
print(f"Vector store created at {persistant_directory}")
print(f"Number of vectors: {VECTOR_STORE._collection.count()}")

C:\Users\samantap\AppData\Local\Temp\ipykernel_9920\3120618224.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  VECTOR_STORE = Chroma(


Vector store created at ./chromaDB
Number of vectors: 65


In [12]:
query="How to work with Node js?"
similar_chunks=VECTOR_STORE.similarity_search(query, k=3)
print(similar_chunks)
similar_chunks_with_score=VECTOR_STORE.similarity_search_with_score(query,k=3)
print(similar_chunks_with_score)# it prints the similar chunks with their scores or ratings

[Document(metadata={'source': 'C:\\Users\\ps19j\\AppData\\Local\\Temp\\tmppjdesij8\\doc_4.txt'}, page_content='signals completion, drastically improving throughput. Promises and async/await syntax further simplify asynchronous code management, reducing callback hell and enhancing maintainability. Node.js excels in microservices, web servers, and real-time applications like chat or streaming services. Its'), Document(metadata={'source': 'C:\\Users\\ps19j\\AppData\\Local\\Temp\\tmpb9s9tz0u\\doc_4.txt'}, page_content='signals completion, drastically improving throughput. Promises and async/await syntax further simplify asynchronous code management, reducing callback hell and enhancing maintainability. Node.js excels in microservices, web servers, and real-time applications like chat or streaming services. Its'), Document(metadata={'source': 'C:\\Users\\samantap\\AppData\\Local\\Temp\\tmpus8vjq_n\\doc_4.txt'}, page_content='signals completion, drastically improving throughput. Promises and

In [13]:
import os
from dotenv import load_dotenv  
load_dotenv()
OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY')


In [14]:
# Using free LLM to know the working of Agentic AI
from openai import OpenAI
client = OpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1"
)

# First API call with reasoning
response = client.chat.completions.create(
  model="x-ai/grok-4.1-fast",
  messages=[
          {
            "role": "user",
            "content": "Can you explain me about Agentic AI?"
          }
        ],
  extra_body={"reasoning": {"enabled": True}}
)
response=response.choices[0].message.content
print(response)




### What is Agentic AI?

Agentic AI refers to a class of artificial intelligence systems designed to act **autonomously like intelligent agents**. These aren't just passive tools that respond to queries (like a basic chatbot); instead, they **perceive their environment, reason about goals, plan actions, and execute them** to achieve objectives with minimal human intervention. Think of them as digital "assistants on steroids" that can handle complex, multi-step tasks in the real world.

#### Key Differences from Traditional/Generative AI
- **Generative AI** (e.g., ChatGPT, DALL-E): Great at creating text, images, or code *in response to prompts*. It's reactive—one input, one output.
- **Agentic AI**: Proactive and **goal-directed**. It can:
  - Break down big goals into sub-tasks.
  - Use tools (e.g., web search, APIs, code execution).
  - Learn from feedback and iterate.
  - Operate in loops: Observe → Plan → Act → Reflect → Repeat.

Analogy: If generative AI is a calculator (you punch

In [15]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(
    model="x-ai/grok-4.1-fast",
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1"
)

response=llm.invoke("What is the capital of France?")
print(response)



content="**Paris** is the capital of France. It's the largest city in the country and has been the political, cultural, and economic center since 987 AD." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 138, 'prompt_tokens': 163, 'total_tokens': 301, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 106, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'x-ai/grok-4.1-fast:free', 'system_fingerprint': None, 'id': 'gen-1764225191-LvadxnABMOd9PMCcEbAl', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--49b3e83f-e7d9-4e9f-ab85-d55c8ef8661e-0' usage_metadata={'input_tokens': 163, 'output_tokens': 138, 'total_tokens': 301, 'input_token_details': {}, 'output_token_details': {'reasoning': 106}}


# Using RAG Chains with Chormadb

In [16]:
# With out using Langchain Expression Language
from langchain_classic.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

retriever=VECTOR_STORE.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4},
)
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001A8F4CF3CB0>, search_kwargs={'k': 4})

In [17]:
# Creating custom prompts
system_prompt="""
Use the following chunks as reference for the answer of the question 
Provide a answer of nearly 200-300 words and use the chunks to gather the answer also use your own knowledge and experience

Context:
{context}

Answer:

"""

prompt=ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)



In [18]:
# Putting in the chain using create_stuff_document_chain-> It will help to combine all the chunks and provide that to the prompt and then that will be passed to the LLM

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nUse the following chunks as reference for the answer of the question \nProvide a answer of nearly 200-300 words and use the chunks to gather the answer also use your own knowledge and experience\n\nContext:\n{context}\n\nAnswer:\n\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001A88CB81E50>, async_client=<openai.resources.chat.completions.complet

#### The total chain is something like this:\n
Input → Format Documents → Create Prompt → LLM Call → Parse Output<br>
**Now what internally happens is that RunnableBinding wraps the whole chain and in that at first it accumulates
the context using format_docs and then it provide that to the ChatPromptTemplate and then it passed it to the ChatOpenAI , the OpenAI provides a response which later showed using StrOutputParser()**

- The format_docs() function is responsible for formatting the documents into a single string.
```
def format_docs(chunks):
return "\n\n".join(chunks.page_content for doc in docs)
```

- The ChatPromptTemplate() function is responsible for creating the prompt for the LLM.
```
ChatPromptTemplate(
    input_variables=['context', 'question'],
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                template='''Use the following chunks as reference for the answer of the question 
Provide a answer of nearly 200-300 words and use the chunks to gather the answer also use your own knowledge and experience

Context:
{context}

Answer:

'''
            )
        ),
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(template='{question}')
        )
    ]
)
```

- The ChatOpenAI() function is responsible for calling the LLM.
```ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, max_tokens=1000)```

- The StrOutputParser() function is responsible for parsing the output of the LLM.
```StrOutputParser(output_key='text')```

- The runnable_binding is responsible for binding the above functions together.
```runnable_binding = RunnableBinding(format_docs, ChatPromptTemplate, ChatOpenAI, StrOutputParser)```

- The runnable is something like this:\n
```runnable = runnable_binding.bind(context=context, question=question)```

- The runnable is something like this:\n
```runnable.run()```


#### Final Rag with both Chunks and Chormadb

In [19]:
rag_chain=create_retrieval_chain(retriever,document_chain)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001A8F4CF3CB0>, search_kwargs={'k': 4}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nUse the following chunks as reference for the answer of the question \nProvide a answer of nearly 200-300 words and use the

In [20]:
# Final chain
# Retriver will retrive based on query from the db -> retreived dataw will passed as context
# -> context will be passed to the LLM
# -> LLM will generate a response based on the custom prompt

last_response=rag_chain.invoke({"input":"I want to know about vector database as a beginner"})
print(last_response)

{'input': 'I want to know about vector database as a beginner', 'context': [Document(metadata={'source': 'C:\\Users\\ps19j\\AppData\\Local\\Temp\\tmppjdesij8\\doc_2.txt'}, page_content='Vector databases have emerged as critical infrastructure for AI-driven applications by enabling the fast, approximate search of high-dimensional embeddings. Unlike traditional relational stores, vector DBs represent data points as mathematical vectors, supporting similarity queries, KNN search, and'), Document(metadata={'source': 'C:\\Users\\ps19j\\AppData\\Local\\Temp\\tmpb9s9tz0u\\doc_2.txt'}, page_content='Vector databases have emerged as critical infrastructure for AI-driven applications by enabling the fast, approximate search of high-dimensional embeddings. Unlike traditional relational stores, vector DBs represent data points as mathematical vectors, supporting similarity queries, KNN search, and'), Document(metadata={'source': 'C:\\Users\\samantap\\AppData\\Local\\Temp\\tmpus8vjq_n\\doc_2.txt'},

In [21]:
last_response['answer']

'### What Are Vector Databases? A Beginner\'s Guide\n\nVector databases have emerged as critical infrastructure for AI-driven applications by enabling the fast, approximate search of high-dimensional embeddings. If you\'re new to this, think of them as a supercharged way to store and search data that\'s not just numbers or text, but **mathematical representations** of things like images, text, or audio.\n\n#### Traditional Databases vs. Vector Databases\nTraditional databases (like SQL ones: MySQL, PostgreSQL) excel at exact matches—e.g., "find users where age = 25." They use tables with rows and columns. But in AI, data is often converted into **vectors** (lists of numbers, like [0.1, 0.5, -0.2, ...]) by models like OpenAI\'s embeddings API or Sentence Transformers. These vectors capture **meaning** or **similarity**—e.g., "king" is close to "queen" in vector space, but far from "apple."\n\nVector DBs (e.g., Pinecone, Weaviate, Milvus, Chroma) store these high-dimensional vectors (oft

'### What is a Vector Database? A Beginner\'s Guide\n\nImagine you\'re trying to find a book in a massive library not by its title or author, but by how similar its content feels to what you\'re thinking—like "stories about brave adventurers in magical worlds." Traditional databases (like SQL ones) excel at exact matches: "Give me the book titled *Harry Potter*." But vector databases are built for **similarity searches**, treating data as points in a high-dimensional space.\n\nAt their core, vector databases store **embeddings**—numerical vectors (arrays of numbers) that represent things like text, images, or audio. For example, AI models like BERT or OpenAI\'s embeddings turn a sentence like "I love cats" into a vector [0.2, -0.5, 1.3, ...] capturing its meaning. These vectors live in hundreds or thousands of dimensions, where similar items cluster close together.\n\nUnlike relational databases with rows and tables, vector DBs use math like **cosine similarity** or **Euclidean distance** to find the **nearest neighbors (KNN)** quickly. They rely on **approximate nearest neighbors (ANN)** algorithms (e.g., HNSW or IVF) for speed on massive datasets—think billions of vectors—without scanning everything.\n\n**Why do they matter?** They\'re the backbone of modern AI apps:\n- **RAG (Retrieval-Augmented Generation)**: ChatGPT-like systems pull relevant docs for accurate answers.\n- **Recommendations**: Netflix suggests shows based on your tastes.\n- **Search & Fraud Detection**: Find similar images or spot anomalies.\n\nPopular open-source/free options: **ChromaDB** (super simple for local use), **Weaviate** (GraphQL-friendly with hybrid search), **Pinecone** (managed cloud service). Getting started? Install Chroma via pip, embed text with Hugging Face, and query in Python—it\'s beginner-friendly!\n\nVector DBs scale with GPU acceleration, distributed indexes, and metadata filtering for real-world apps. They\'re exploding in AI because they make "fuzzy" searches lightning-fast. Dive in with a tutorial; you\'ll see why they\'re essential infrastructure!\n\n*(Word count: 278)*'

In [23]:
# Chain using Langchain Expression Language
# In this chain no retriver is used directly its just for learning purpose how langchain expression language works

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel,RunnableBinding,RunnablePassthrough
llm=ChatOpenAI(
    model="x-ai/grok-4.1-fast",
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1"
)
prompt=ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant with name {name}"),
        ("human", "{query}"),
    ]
)
chain = prompt | llm | StrOutputParser()
result = chain.invoke({"name": "RAG-Bot", "query": "Future of Teoco Company?"})
print(result)

### Future Outlook for TEOCO Corporation

TEOCO (The Evans & Company) is a leading provider of big data analytics, AI-driven software, and services primarily for the telecommunications industry. Founded in 1995 and headquartered in Fairfax, Virginia, USA, the company specializes in revenue management, network optimization, fraud detection, and customer experience solutions. It's privately held, serves over 300 telecom operators globally (including giants like Verizon, Vodafone, and AT&T), and has grown through organic expansion and acquisitions (e.g., Asterisk in 2019 for customer analytics).

#### Key Strengths Positioning TEOCO for Growth
- **Industry Tailwinds**: 
  - Telecoms are investing heavily in 5G rollout, edge computing, Open RAN, and IoT. TEOCO's platforms like mTOP (network optimization) and RAVE (revenue assurance) are well-suited for these, using AI/ML to reduce costs and improve efficiency.
  - Global data explosion and regulatory pressures (e.g., spectrum auctions, sus

In [27]:
from langchain_core.prompts import ChatPromptTemplate
# Define docs 
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

custom_prompt=ChatPromptTemplate.from_template(
    """
    Use the following chunks as reference for the answer of the question 
    Provide a answer of nearly 200-300 words and use the chunks to gather the answer also use your own knowledge and experience

    Context:
    {context}

    Answer:
    """
)

In [28]:
# LCEL with RAG
rag_chain=(
{"context":retriever|format_docs,"question": RunnablePassthrough() }
| custom_prompt
| llm
| StrOutputParser()
)
rag_chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001A8F4CF3CB0>, search_kwargs={'k': 4})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n    Use the following chunks as reference for the answer of the question \n    Provide a answer of nearly 200-300 words and use the chunks to gather the answer also use your own knowledge and experience\n\n    Context:\n    {context}\n\n    Answer:\n    '), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001A88DD6C510>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001A88CCCD6D0>, root_

In [30]:
response_lcel=rag_chain.invoke("How to optimize SQL queries?")
print(response_lcel)

### Mastering SQL Query Optimization for Scalable Databases

SQL query optimization is crucial for scalable database operations, ensuring efficient performance as data volumes grow. One foundational strategy is indexing frequently searched columns, such as those in WHERE clauses, ORDER BY, or JOIN conditions. Indexes act like a book's index, allowing the database engine to quickly locate data without full table scans. However, balance is key: too many indexes can degrade write performance during INSERT, UPDATE, or DELETE operations, as each modification requires updating multiple index structures, increasing I/O overhead and storage needs.

To optimize queries effectively, always avoid 'SELECT *'—it fetches unnecessary columns, bloating data transfer and memory usage. Instead, explicitly select only required fields (e.g., `SELECT user_id, name FROM users`). Minimize joins to essential tables; excessive multi-table joins can lead to Cartesian products or slow nested loop executions. Pri

#### Adding new Document into the Vector Store
- Best use case when database is updated or new data came and we want to update the ai assistant

In [31]:
sample_date="""
About Reinforcement Learning with Expert Iteration (RLEI)
Reinforcement Learning with Expert Iteration (RLEI) is an advanced machine learning technique that combines reinforcement learning with expert knowledge to enhance decision-making processes. In RLEI, an agent learns to make optimal decisions by interacting with an environment, receiving feedback in the form of rewards or penalties. The unique aspect of RLEI is the integration of expert demonstrations, which guide the learning process and help the agent explore more effectively.
These demonstrations provide valuable insights into successful strategies, allowing the agent to learn from human expertise while still leveraging the power of reinforcement learning algorithms. RLEI has shown promise in complex domains such as game playing, robotics, and autonomous systems, where expert knowledge can significantly accelerate the learning process and improve overall performance.
The technique involves iteratively refining the agent's policy by alternating between learning from expert demonstrations and self-improvement through exploration. This dual approach enables the agent to benefit from both human expertise and autonomous learning, leading to more robust and efficient decision-making capabilities.
"""

In [32]:
new_doc=Document(
    page_content=sample_date,
    metadata={"source":"manual entry"}
)

In [33]:
spilted_new_docs=splitter.split_documents([new_doc])
print("Total new chunks:", len(spilted_new_docs))

Total new chunks: 5


In [35]:
VECTOR_STORE.add_documents(spilted_new_docs)
print("New document added to the vector store.")
print("Total vectors after addition:", VECTOR_STORE._collection.count())

New document added to the vector store.
Total vectors after addition: 75


In [37]:
rag_chain.invoke("What you know about Reinforcement Learning with Expert Iteration?")

"**What is Reinforcement Learning with Expert Imitation (RLEI)?**\n\nReinforcement Learning with Expert Imitation (RLEI) is an advanced hybrid machine learning paradigm that combines the strengths of imitation learning and traditional reinforcement learning (RL) to accelerate policy optimization in complex environments. By incorporating expert demonstrations—such as human or high-performing agent trajectories—RLEI provides agents with valuable insights into successful strategies from the outset, significantly speeding up the learning process and enhancing overall performance compared to pure RL methods like Q-learning or PPO, which often suffer from sparse rewards and sample inefficiency.\n\nThe core technique revolves around an iterative refinement loop. The agent alternates between two phases: (1) **imitation from experts**, where it learns to mimic optimal behaviors via methods like behavioral cloning or generative adversarial imitation learning (GAIL), distilling human expertise in

### Advanced RAG Technique - Using Conversational Memory
- create_history_aware_retirver is used to record the conversation of the prevous hisotory it helps to track that so that all conversation occurs in sync
- MessagePlaceholder - It is used to placeholder for history in the chat hostory in prompts
- HumanMessage - Standard Message for conversation history

In [ ]:
## creating the prompt that using the chat history
